In [1]:
import pandas as pd
import numpy as np
# import guidedlda
import time

from sklearn.feature_extraction.text import CountVectorizer

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import string

"""BQ imports"""
from google.cloud import bigquery
client = bigquery.Client()

from itertools import chain
from collections import Counter

In [2]:
train = pd.read_pickle("./data/preprocess_mnet_train.pkl")

train.drop_duplicates('natid', keep='first', inplace=True)
train.reset_index(drop=True, inplace=True)

print(train.shape)
train.head()

(10078, 8)


,natid,pub_date,category,tier1,tier2,summary,clean_summary,label
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7
1,blogandpostid/blog/post/4587-60d354bc9690ca000...,2021-06-23,Food and Drink,Travel,Travel Type,Celebrate The Lodge At Woodloch’s 15-Year Anni...,celebrate lodge woodloch's 15-year anniversary...,23
2,blogandpostid/blog/post/3113-5f526f97559f59000...,2020-09-04,Music and Audio,Music and Audio,Hip Hop Music,Cardi B And Megan Thee Stallion Both Earn Thei...,cardi b megan thee stallion earn no. 1 hit you...,12
3,blogandpostid/blog/post/5259-6015f002c7824f000...,2021-02-09,Travel,Shopping,Gifts and Greetings Cards,Valentine’s Day Gift Guide: The Most Romantic ...,valentine's day gift guide: romantic hotel pac...,13
4,blogandpostid/blog/post/4983-5f8064d69a4142000...,2020-10-14,Shopping,Shopping,Sales and Promotions,The Best Prime Day Bike Deals For Saving Big o...,best prime day bike deals saving big bicycles ...,13


In [3]:
train.tier1.value_counts()

Business and Finance        445
Personal Finance            442
News and Politics           442
Technology & Computing      440
Sports                      438
Medical Health              436
Video Gaming                422
Travel                      414
Science                     406
Automotive                  404
Style & Fashion             403
Events and Attractions      402
Careers                     398
Shopping                    395
Food & Drink                392
Music and Audio             388
Real Estate                 381
Education                   376
Movies                      369
Television                  343
Healthy Living              339
Home & Garden               328
Hobbies & Interests         314
Family and Relationships    302
Fine Art                    226
Pets                        152
Books and Literature        148
Pop Culture                 133
Name: tier1, dtype: int64

In [4]:
test = pd.read_pickle("./data/preprocess_mnet_test.pkl")
test.drop_duplicates('natid', keep='first', inplace=True)
test.reset_index(drop=True, inplace=True)

print(test.shape)
test.head()

(1258, 8)


,natid,pub_date,category,tier1,tier2,summary,clean_summary,label
0,blogandpostid/blog/post/3541-60eb91d76f189c000...,2021-07-12,Movies,Movies,Indie and Arthouse Movies,What Will The Movie Industry Look Like After C...,movie industry look like covid?. movie industr...,2
1,blogandpostid/blog/post/7380-5f7a952000e768000...,2020-10-05,Video Gaming,Video Gaming,Video Game Genres,A YouTuber Made An ‘Among Us’ Mode In ‘Overwat...,youtuber us' mode overwatch' looks fantastic. ...,10
2,blogandpostid/blog/post/50373-5fcfe6475a1f9400...,2020-12-08,News and Politics,News and Politics,Politics,Rudy Giuliani Says He’s ‘Doing Fine’ With Covi...,"rudy giuliani says fine' covid, hopes released...",0
3,blogandpostid/blog/post/25-607d3d189d9fd30006d...,2021-04-19,Technology and Computing,Personal Finance,Personal Investing,Will Apple Stock’s Q2 Results Beat Consensus?....,apple stock's q2 results beat consensus?. appl...,21
4,blogandpostid/blog/post/6937-60b6f3feff8c53000...,2021-06-02,Movies,Movies,Documentary Movies,Ava DuVernay’s Array Film Collective Partners ...,ava duvernay's array film collective partners ...,2


In [5]:
test.tier1.value_counts()

News and Politics           50
Personal Finance            50
Technology & Computing      50
Medical Health              50
Events and Attractions      50
Education                   50
Careers                     50
Sports                      50
Real Estate                 50
Travel                      50
Video Gaming                50
Business and Finance        50
Movies                      49
Television                  49
Healthy Living              49
Music and Audio             49
Science                     49
Shopping                    49
Automotive                  49
Style & Fashion             49
Family and Relationships    49
Food & Drink                48
Hobbies & Interests         48
Home & Garden               47
Fine Art                    25
Pets                        17
Books and Literature        17
Pop Culture                 15
Name: tier1, dtype: int64

In [6]:
import re
import html.parser
import unicodedata

# --- DEFINE VARIABLES ---

contraction_dict_file = './data/contraction_dictionary.pkl'
# Load Contraction Dictionary
c_dict = pd.read_pickle(contraction_dict_file)
# Compile the contraction dict
c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))


# --- DEFINE FUNCTIONS ---

# Function to expand contractions
def expandContractions(text, c_re=c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)

# Function to remove unicode
def normalize_unicode(input_str):
    '''
    replaces accents by converting to byte literal and then decoding to utf-8
    '''
    nfkd_form = unicodedata.normalize('NFKD', input_str) 
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode("utf-8")  

# Function to remove the html
class HTMLTextExtractor(html.parser.HTMLParser):
    def __init__(self):
        super(HTMLTextExtractor, self).__init__()
        self.result = []
    def handle_data(self, d):
        self.result.append(d)
    def get_text(self):
        return ''.join(self.result)

def html_to_text(html):
    s = HTMLTextExtractor()
    s.feed(html)
    return s.get_text()

def remove_html(text):
    text = str(text)
    string = re.sub('<[^<]+?>', ' ', text)
    string = html_to_text(string)
    string = re.sub('’', "'", string) 
    # [^\x00-\x7F]+ searches for all non-ASCII values
    string = re.sub(r'[^\x00-\x7F]+', ' ', string)
    # below escapes special characters mentioned inside []
    string = re.sub(r'\[.*?\]', ' ', string) 
    string = string.replace('\r', '  ')
    string = string.replace('\n', '  ')
    string = re.sub(' +', ' ', string)
    return string

def remove_newline(string):
    #remove carriage return
    string = string.replace('\r', '') 
    #remove newline
    return string.replace('\n', '') 

* Pre-process article text

In [7]:
stopwords_list = stopwords.words('english')
punctuations = list(set(string.punctuation))
unwanted_list = punctuations + stopwords_list

def clean_df(df):
    '''
    Applies above text cleaning functions and returns a df with cleaned text
    '''

    # Remove html, normalize unicode
    df['clean_body'] = df['summary'].astype(str).str.lower()
    df['clean_body'] = df['clean_body'].apply(remove_newline).astype(str)
    df['clean_body'] = df['clean_body'].apply(expandContractions, c_re=c_re)
    df['clean_body'] = df['clean_body'].apply(normalize_unicode)
    df['clean_body'] = df['clean_body'].apply(remove_html).str.strip()
    df['clean_body'] = df['clean_body'].apply(lambda x: re.sub(r"<[^>]*>", " ", x)) 
    df['clean_body'] = df['clean_body'].apply(lambda text: ' '.join([x.lower() for x in word_tokenize(text) if x.lower() not in unwanted_list and len(x)>1]))
    
    # remove numbers
    df['clean_body'] = df['clean_body'].apply(lambda text: re.sub('[0-9]+', '', text))
    # remove more than 1 space in between words 
    df['clean_body'] = df['clean_body'].replace('\s+', ' ', regex=True) 
    
    return df

In [8]:
# clean train

train = clean_df(train)
train.shape #2mins 

(10078, 9)

In [9]:
# clean test

test = clean_df(test)
test.shape

(1258, 9)

In [13]:
from nltk import pos_tag

# Contraction dictionary
STOP_WORDS = pd.read_pickle('./data/forbes_stop_words.pkl')

lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = nltk.word_tokenize(text)
    text = [lemmatizer.lemmatize(x) for x in text]
    text = [x for x in text if len(x) > 1]
    
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    # tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(text) if is_noun(pos)] 
    
    # return text
    return ' '.join(all_nouns)

In [11]:
# process-noun train

train['processed_noun'] = train['clean_body'].apply(clean_text) #3mins

In [12]:
# process-noun test

test['processed_noun'] = test['clean_body'].apply(clean_text) 

In [14]:
train.clean_summary.str.len().describe().astype(int)

count    10078
mean      3592
std       2133
min        433
25%       2227
50%       3149
75%       4326
max      30344
Name: clean_summary, dtype: int64

In [15]:
train.processed_noun.str.len().describe().astype(int)

count    10078
mean      1727
std       1034
min        210
25%       1058
50%       1516
75%       2105
max      14969
Name: processed_noun, dtype: int64

* **TF_IDF for every article**

* I am doing fit_transform on train and transform on test. Then taking top 10 words in both sets. Why?
    * https://kavita-ganesan.com/extracting-keywords-from-text-tfidf/#.YVXbZprMLIU (Doing exactly what we are doing)
     
    * My understanding - 
        * tf = term freq. idf = weighting factor. 
        * On train set:
            * tfidf calculates tf. Suppose "forbes" high freq and "sun" had lower freq. 
            * idf will give lower weight to "forbes" and higher to "sun". Similarly it will learn weights for all vocab (words)
            * in our use-case, we will take out top 10 important words for each article like "sun"
            * model will learn relationships on train data like sun (which idf gave more importance to) appears in Space category/class
        * On test set:
            * tfidf calculates tf. Again suppose "forbes" high freq and "sun" had lower freq
            * tfidf.transform() has learned that sun has more weight in train set. So, it will assign similar weights to test set
            * classification model will see top 10 word in article has sun. so it will predict Space for that article
    * More references: 
        * [Ref 1](https://stackoverflow.com/questions/47778403/computing-tf-idf-on-the-whole-dataset-or-only-on-training-data) [Ref 2](https://stackoverflow.com/questions/59950022/should-tfidf-be-generated-with-train-and-test-sets-combined)

In [16]:
train.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,clean_body,processed_noun
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7,wework accept cryptoand pay landlords crypto t...,wework cryptoand pay landlord crypto topline w...


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create the tfidf weights
tfidf = TfidfVectorizer(
    min_df=2,
    max_df=0.95,
    max_features=5000
)

texts = train.clean_body.to_list()

X_tfidf = tfidf.fit_transform(texts).toarray()

vocab = tfidf.vocabulary_

reverse_vocab = {v:k for k,v in vocab.items()}

feature_names = tfidf.get_feature_names()

idx = X_tfidf.argsort(axis=1)

tfidf_max10 = idx[:,-10:]

train['top10_tfidf'] = [[reverse_vocab.get(item) for item in row] for row in tfidf_max10 ]

In [19]:
train['top10_tfidf'] = train['top10_tfidf'].apply(lambda x: ', '.join(map(str, x)))

In [23]:
cols_to_keep = ['natid', 'pub_date', 'category', 'tier1', 'tier2', 'summary', 'clean_summary', 'label', 'top10_tfidf']

train[cols_to_keep].to_csv("./data/mnet_train_tfidf.csv", index=False)

train[cols_to_keep].to_pickle("./data/mnet_train_tfidf.pkl")

In [25]:
test_txt = test.clean_body.to_list()

X_test_tfidf = tfidf.transform(test_txt).toarray()
print(X_test_tfidf.shape)

idx_test = X_test_tfidf.argsort(axis=1)
print(idx_test.shape)

tfidf_max10_test = idx_test[:,-10:]

(1258, 5000)
(1258, 5000)


In [26]:
test['top10_tfidf'] = [[reverse_vocab.get(item) for item in row] for row in tfidf_max10_test ] 

test['top10_tfidf'] = test['top10_tfidf'].apply(lambda x: ', '.join(map(str, x)))

In [28]:
test[cols_to_keep].to_csv("./data/mnet_test_tfidf.csv", index=False)

test[cols_to_keep].to_pickle("./data/mnet_test_tfidf.pkl")

In [30]:
train[cols_to_keep]

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label,top10_tfidf
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7,"billion, announced, landlords, accept, tesla, ..."
1,blogandpostid/blog/post/4587-60d354bc9690ca000...,2021-06-23,Food and Drink,Travel,Travel Type,Celebrate The Lodge At Woodloch’s 15-Year Anni...,celebrate lodge woodloch's 15-year anniversary...,23,"celebrate, pink, cocktail, treatments, resorts..."
2,blogandpostid/blog/post/3113-5f526f97559f59000...,2020-09-04,Music and Audio,Music and Audio,Hip Hop Music,Cardi B And Megan Thee Stallion Both Earn Thei...,cardi b megan thee stallion earn no. 1 hit you...,12,"frames, track, highest, first, reached, countr..."
3,blogandpostid/blog/post/5259-6015f002c7824f000...,2021-02-09,Travel,Shopping,Gifts and Greetings Cards,Valentine’s Day Gift Guide: The Most Romantic ...,valentine's day gift guide: romantic hotel pac...,13,"night, couples, private, weekend, enjoy, roman..."
4,blogandpostid/blog/post/4983-5f8064d69a4142000...,2020-10-14,Shopping,Shopping,Sales and Promotions,The Best Prime Day Bike Deals For Saving Big o...,best prime day bike deals saving big bicycles ...,13,"lights, visible, kids, safer, amazon, day, ele..."
...,...,...,...,...,...,...,...,...,...
10073,blogandpostid/blog/post/50165-60b799ae8dc0df00...,2021-06-02,Business and Finance,Careers,Remote Working,NatWest Expects Just 13% Of Staff To Return To...,natwest expects 13% staff return office full-t...,22,"role, fits, split, expects, reuters, rose, emp..."
10074,blogandpostid/blog/post/4208-5f6fd04a232b58000...,2020-09-26,Science,Science,Environment,Extreme Heat And Extreme Drought Are Occurring...,extreme heat extreme drought occurring terrify...,1,"regions, intense, fires, frequency, hot, tempe..."
10075,blogandpostid/blog/post/50560-5fa5571582d5ee00...,2020-11-06,Medical Health,Medical Health,Diseases and Conditions,What You Need To Know About The Swine Flu Dete...,need know swine flu detected canada. good reas...,16,"affect, genetic, people, found, variant, birds..."
10076,blogandpostid/blog/post/1961-60c23c6e304792000...,2021-06-13,Fine Art,Fine Art,Fine Art Photography,"25 Outstanding Artistic Photos, Winners Of Int...","25 outstanding artistic photos, winners intern...",24,"life, explains, creative, nature, awards, lake..."


In [ ]:
# BELOW WAS WRONG DIRECTION

* **Tier 1 labels with TF_IDF**
    * For each class - find most distinctive TF-IDF based top 10 words
        * Manually tweaked a bit because LDA needs good quality words to be able to group words together into topics. 

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

output = {}

for t1 in list(sorted(df.tier1.unique())):
    texts = df[df.tier1==t1].processed_noun.to_list()
    
    # Create the tfidf weights
    tfidf_vectorizer = TfidfVectorizer(
        min_df=2,
        max_df=0.95,
        max_features=5000,
        ngram_range=(1, 2),
    #     preprocessor=' '.join
    )

    tfidf = tfidf_vectorizer.fit_transform(texts)

    # Save the feature names
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
    
    feature_array = np.array(tfidf_vectorizer.get_feature_names())
    tfidf_sorting = np.argsort(tfidf.toarray()).flatten()[::-1]

    n = 20
    top_n = feature_array[tfidf_sorting][:n]
    output[t1] = top_n
    
tfidf_noun = pd.DataFrame(output)

In [79]:
tfidf_noun[tfidf_noun.columns[9:]]

,Healthy Living,Hobbies & Interests,Home & Garden,Medical Health,Movies,Music and Audio,News and Politics,Personal Finance,Pets,Pop Culture,Real Estate,Science,Shopping,Sports,Style & Fashion,Technology & Computing,Television,Travel,Video Gaming
0,influencers,story,dyson,influenza,reporter,act,hart,advisor,guam,rating,income,heat,deal,playoff,skin,xt,advantage,music,hitman
1,micro,organization,field,virus,thomas,space list,house,firm,population,viewer,property,drought,card,tampa,com,rx,factor,yacht,problem xbox
2,uncle,truth,season,hn,sundance,month,house administration,model,release,adult,utility,heat drought,amazon,tampa bay,oil,rx xt,team,space,xbox playstation
3,business,business,pirate,bird,camera,album,iowa,independence,depression,cbs,value,frequency,day,goal,acid,amd,mode,house,playstation pc
4,sister,parent,league,virus virus,fire,title,recount,landscape,snake,harry cbs,renovation,temperature,price,bay,serum,hardware,momentum,tech,server
5,marketing,share story,join,people,news,copy,effort,bank,bird,time period,increase,wildfire,money day,game,hydration,hardware news,concept,brand,suit
6,ad,failure,dysons,variant,print,swift,lawsuit,continuum,dr,game,unit,vegetation,gift card,everybody,mask,pc hardware,stadium,party,couldnt
7,store,storyteller,robert,gene,physicality,claim,case house,category,zoo,cbs primetime,property value,record,membership,effort,pore,news review,home,event,problem
8,instagram,institution,base,flu,india,cd,committee,wealth,conservation,primetime,tenant,state event,subscription,proud,barrier,twitter facebook,game,mission,progress
9,australia,story client,guy,host,space,isnt,election,transition,island,daniel,expense,extreme heat,youre,hurricane,treatment,announces,share,artist,moment


In [86]:
new_t1_label = {}

new_t1_label[tfidf_noun.columns[0]] = re.sub('and', '&', tfidf_noun.columns[0]) + ' ' + 'auto sale car electric vehicle'
new_t1_label[tfidf_noun.columns[1]] = re.sub('and', '&', tfidf_noun.columns[1]) + ' ' + 'author storyteller fiction read biography'
new_t1_label[tfidf_noun.columns[2]] = re.sub('and', '&', tfidf_noun.columns[2]) + ' ' + 'consumer marketing yield quarter capital'
new_t1_label[tfidf_noun.columns[3]] = re.sub('and', '&', tfidf_noun.columns[3]) + ' ' + 'role employee interview skill job'
new_t1_label[tfidf_noun.columns[4]] = re.sub('and', '&', tfidf_noun.columns[4]) + ' ' + 'college degree enrollment university campus'
new_t1_label[tfidf_noun.columns[5]] = re.sub('and', '&', tfidf_noun.columns[5]) + ' ' + 'celebration party parks museum auction'
new_t1_label[tfidf_noun.columns[6]] = re.sub('and', '&', tfidf_noun.columns[6]) + ' ' + 'child mother care family custody'
new_t1_label[tfidf_noun.columns[7]] = re.sub('and', '&', tfidf_noun.columns[7]) + ' ' + 'artist photography digital art studio'
new_t1_label[tfidf_noun.columns[8]] = re.sub('and', '&', tfidf_noun.columns[8]) + ' ' + 'wine taste cuisine menu restaurant'
new_t1_label[tfidf_noun.columns[9]] = re.sub('and', '&', tfidf_noun.columns[9]) + ' ' + 'wellness fitness exercise nutrition mindfulness'

In [87]:
new_t1_label[tfidf_noun.columns[10]] = re.sub('and', '&', tfidf_noun.columns[10]) + ' ' + 'craft puzzle instrument' # hobbies & interest - not great words
new_t1_label[tfidf_noun.columns[11]] = re.sub('and', '&', tfidf_noun.columns[11]) + ' ' + 'interior outdoor remodel appliances renovation'
new_t1_label[tfidf_noun.columns[12]] = re.sub('and', '&', tfidf_noun.columns[12]) + ' ' + 'virus vaccine patient disease outbreak'
new_t1_label[tfidf_noun.columns[13]] = re.sub('and', '&', tfidf_noun.columns[13]) + ' ' + 'award film theater release documentary'
new_t1_label[tfidf_noun.columns[14]] = re.sub('and', '&', tfidf_noun.columns[14]) + ' ' + 'album title song chart band'
new_t1_label[tfidf_noun.columns[15]] = re.sub('and', '&', tfidf_noun.columns[15]) + ' ' + 'administration lawsuit election opposition bill'
new_t1_label[tfidf_noun.columns[16]] = re.sub('and', '&', tfidf_noun.columns[16]) + ' ' + 'bank wealth savings tax loan'
new_t1_label[tfidf_noun.columns[17]] = re.sub('and', '&', tfidf_noun.columns[17]) + ' ' + 'breeding species extinction animal bird'
new_t1_label[tfidf_noun.columns[18]] = re.sub('and', '&', tfidf_noun.columns[18]) + ' ' + 'primetime celebrity scandal rating famous'
new_t1_label[tfidf_noun.columns[19]] = re.sub('and', '&', tfidf_noun.columns[19]) + ' ' + 'property rent listing tenant mortgage'

In [88]:
new_t1_label[tfidf_noun.columns[20]] = re.sub('and', '&', tfidf_noun.columns[20]) + ' ' + 'extreme temperature scientist space biology'
new_t1_label[tfidf_noun.columns[21]] = re.sub('and', '&', tfidf_noun.columns[21]) + ' ' + 'deal subscription gift promotion holiday'
new_t1_label[tfidf_noun.columns[22]] = re.sub('and', '&', tfidf_noun.columns[22]) + ' ' + 'playoff coach athletes tournament league'
new_t1_label[tfidf_noun.columns[23]] = re.sub('and', '&', tfidf_noun.columns[23]) + ' ' + 'skincare beauty designer clothing trends'
new_t1_label[tfidf_noun.columns[24]] = re.sub('and', '&', tfidf_noun.columns[24]) + ' ' + 'hardware electronics app server computing' 
new_t1_label[tfidf_noun.columns[25]] = re.sub('and', '&', tfidf_noun.columns[25]) + ' ' + 'adaptation show series drama episode'
new_t1_label[tfidf_noun.columns[26]] = re.sub('and', '&', tfidf_noun.columns[26]) + ' ' + 'resort airport hotel flight beach'
new_t1_label[tfidf_noun.columns[27]] = re.sub('and', '&', tfidf_noun.columns[27]) + ' ' + 'console pc playstation xbox nintendo'

In [89]:
new_t1_label

{'Automotive': 'Automotive auto sale car electric vehicle',
 'Books and Literature': 'Books & Literature author storyteller fiction read biography',
 'Business and Finance': 'Business & Finance consumer marketing yield quarter capital',
 'Careers': 'Careers role employee interview skill job',
 'Education': 'Education college degree enrollment university campus',
 'Events and Attractions': 'Events & Attractions celebration party parks museum auction',
 'Family and Relationships': 'Family & Relationships child mother care family custody',
 'Fine Art': 'Fine Art artist photography digital art studio',
 'Food & Drink': 'Food & Drink wine taste cuisine menu restaurant',
 'Healthy Living': 'Healthy Living wellness fitness exercise nutrition mindfulness',
 'Hobbies & Interests': 'Hobbies & Interests craft puzzle instrument',
 'Home & Garden': 'Home & Garden interior outdoor remodel appliances renovation',
 'Medical Health': 'Medical Health virus vaccine patient disease outbreak',
 'Movies': '

In [91]:
import joblib

joblib.dump(new_t1_label, "new_t1_label.joblib")

['new_t1_label.joblib']

* **Tier 1 labels with keyphrases**

In [7]:
sql= """
    SELECT 
        * 
    FROM 
        `api-project-901373404215.keyphrase_extraction.forbes_keyphrases` 
    """

# Send the query to the api and return a df
keyphrase_df = client.query(sql).to_dataframe()

print("Shape: ", keyphrase_df.shape) 

Shape:  (77287, 4)


In [9]:
keyphrase_df.head(1)

,naturalid,publish_timestamp,title,keyphrases
0,blogandpostid/blog/post/7082-6152ddecc7e051000...,2021-09-28 07:49:36+00:00,Fat Bear Week Is Back! Here Is The 2021 Bracke...,"[bonus bears, fat bear week champion, fat bear..."


In [10]:
df.shape

(10078, 8)

In [12]:
df[df.natid.isin(keyphrase_df.naturalid)].shape

(6171, 8)

In [13]:
df.head(1)

,natid,pub_date,category,tier1,tier2,summary,clean_summary,label
0,blogandpostid/blog/post/50808-607eda2d97a5b000...,2021-04-20,Personal Finance,Real Estate,Real Estate Renting and Leasing,WeWork To Accept Crypto—And Will Pay Its Landl...,wework accept crypto pay landlords crypto. top...,7


In [14]:
keyphrase_df.sort_values('publish_timestamp')

,naturalid,publish_timestamp,title,keyphrases
73622,blogandpostid/blog/post/50453-5fdb887647c7ec00...,2020-12-17 12:02:39+00:00,How One Company Is Redefining Hiring And Worki...,"[team members, lower employment rates, employm..."
53057,blogandpostid/blog/post/50686-5fda3fc525810c00...,2020-12-17 12:11:26+00:00,Peach Tree Rascals: How To Succeed Without Eve...,"[peach tree rascals, later success, american a..."
59780,blogandpostid/blog/post/4613-5fdb8fd632770e000...,2020-12-17 12:12:26+00:00,Air Travel Will Soar Again — Here’s What It Wi...,"[air travel, aircraft air filtration systems, ..."
17726,blogandpostid/blog/post/1929-5fdb7c6d47c7ec000...,2020-12-17 12:18:44+00:00,Helicopters Removed From ‘Call Of Duty: Warzon...,"[black ops cold war, duty warzone, new attack ..."
68730,blogandpostid/blog/post/4330-5fdb90c4df02ed000...,2020-12-17 12:18:59+00:00,Emirates Airline Writes Off $600 Million In Fu...,"[fuel hedging costs, icds airline subsidiaries..."
...,...,...,...,...
4,blogandpostid/blog/post/2696-6152faa878bed1000...,2021-09-28 07:35:59+00:00,Will The Global Energy Crunch Hit U.S. Natural...,"[natural gas prices, natural gas consumption, ..."
3,blogandpostid/blog/post/7141-615223b5b2e0ac000...,2021-09-28 07:45:00+00:00,How To Get Your Startup Noticed In The Sea Of ...,"[startups, funding hesitation, funding records..."
2,blogandpostid/blog/post/7037-61527218de12b3000...,2021-09-28 07:45:00+00:00,OKC Thunder Media Day Themes And Final Trainin...,"[final training camp roster, okc thunder media..."
1,blogandpostid/blog/post/4139-6151d5e426eb37000...,2021-09-28 07:45:00+00:00,How To Make Your PR & Communications Strategie...,"[small data, data literacy, data culture, big ..."


In [65]:
sql= """
    SELECT * FROM (
            SELECT 
                LOWER(naturalId) AS iab_natid, 
                RANK() OVER (PARTITION BY naturalid ORDER BY timestamp DESC) AS mostrecent,
                tier1, 
                tier2,
                categoryName
            FROM `api-project-901373404215.Content.mnet_iab_categories` where tier1 IS NOT NULL
            ) 
        WHERE mostrecent = 1 
    """

# Send the query to the api and return a df
mnet_iab = client.query(sql).to_dataframe()

print("Shape: ", mnet_iab.shape) 

Shape:  (172459, 5)


In [68]:
subset = pd.merge(mnet_iab, 
                  keyphrase_df[['naturalid', 'keyphrases']], 
                  how='inner', 
                  left_on='iab_natid', right_on='naturalid'
                 )
subset.shape

(71073, 7)

In [69]:
subset.tier1.value_counts()

Business and Finance        18264
Personal Finance             7045
News and Politics            6062
Technology & Computing       6046
Sports                       5504
Medical Health               3904
Travel                       2577
Video Gaming                 2064
Style & Fashion              1719
Automotive                   1621
Television                   1601
Food & Drink                 1575
Music and Audio              1542
Events and Attractions       1395
Careers                      1388
Education                    1310
Science                      1251
Real Estate                  1149
Shopping                     1105
Movies                       1084
Home & Garden                 563
Healthy Living                539
Hobbies & Interests           441
Family and Relationships      406
Books and Literature          259
Pop Culture                   240
Fine Art                      191
Pets                          151
Religion & Spirituality        47
Content Channe

In [70]:
subset.head(1)

,iab_natid,mostrecent,tier1,tier2,categoryName,naturalid,keyphrases
0,blogandpostid/blog/post/1174-608028f897a5b0000...,1,Video Gaming,PC Games,PC Games,blogandpostid/blog/post/1174-608028f897a5b0000...,"[video content, video pass, leaked playstation..."


In [71]:
subset_bf = subset[subset.tier1=='Automotive']

In [72]:
bf_all = list(chain(*subset_bf.keyphrases))
frequency = Counter(bf_all)

In [37]:
pd.DataFrame.from_dict(dict(frequency), orient='index').reset_index().rename(columns={'index':'phrase', 0:'freq'}).sort_values('freq', ascending=False).head(10)

,phrase,freq
9,business owners,14
20,business leaders,12
69,successful public relations,8
854,media strategy,7
276,new products,6
462,small businesses,5
86,company values,5
269,new customers,4
1058,new technologies,4
24,employee experience,4


In [46]:
pd.DataFrame.from_dict(dict(frequency), orient='index').reset_index().rename(columns={'index':'phrase', 0:'freq'}).sort_values('freq', ascending=False).head(10)

,phrase,freq
523,new features,6
1290,virtual reality,6
1213,business leaders,5
507,social media platforms,5
65,iphone users,4
348,cloud computing,4
272,remote work,4
107,ransomware attacks,4
1152,new data,3
852,new products,3


In [51]:
pd.DataFrame.from_dict(dict(frequency), orient='index').reset_index().rename(columns={'index':'phrase', 0:'freq'}).sort_values('freq', ascending=False).head(10)

,phrase,freq
4,electric vehicles,22
42,semi-autonomous cars,14
0,electric cars,13
48,level and level true self-driving vehicles,13
348,driving actions,11
192,vehicles,10
659,auto sales,9
878,battery electric vehicles,9
340,human drivers,8
46,todays ai driving systems,8
